In [1]:
import gymnasium
import pandas as pd
import json
import numpy as np
from models.models import DRLAgent
from ray.air.config import RunConfig, ScalingConfig
from ray import tune
from ray.tune import Callback
from ray.train.rl import RLTrainer
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.multi_agent.hrl import HRL
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import MedianStoppingRule, HyperBandScheduler, AsyncHyperBandScheduler
import time
from ray.tune import CLIReporter
from models.callbacks import RayTuneCustomTradesLoggingCallback
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.search import ConcurrencyLimiter
import collections
import logging
import ray
from ray.tune.registry import register_env
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.a2c import A2CConfig
from ray.rllib.algorithms import a2c
import ray.air as air
from configs.load_specific_data import pharma_basket
from ray.tune.logger import pretty_print
import warnings
warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
import os
os.environ['PYTHONWARNINGS'] = 'ignore'
from ray.rllib.algorithms import ppo, a2c
from preprocessors.data_cleaner import forward_fill_missing_stock_data
ray.init(_temp_dir='/Users/floriankockler/rayresults/')

# from ray.rllib.algorithms.a2c.a2c import A2CConfig, A2C

2023-08-31 20:49:29,191	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-31 20:49:31,687	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.6.1
Dashboard:,http://127.0.0.1:8265


In [2]:
train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()



worker_config =   {"hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  }
manager_config = {
    "n_stocks": stock_dimension, 
}
config={


        "worker_config" : worker_config,
        "manager_config": manager_config
        }

env = HRL(config)
env.observation_space.sample()

_initiate_state: {'manager': array([66])}
self.state -manager init {'manager': array([66])}


OrderedDict([('manager', OrderedDict([('manager', array([30], dtype=int32))])),
             ('worker',
              array([ 0.00773068,  1.0449815 ,  0.31078532,  0.1009679 ,  0.3833309 ,
                     -1.8681492 ,  1.4080025 , -0.62053263,  0.7398014 , -0.5355283 ,
                     -0.9006054 , -0.08138815, -0.5155712 , -0.03982062,  0.4417007 ,
                     -0.08708782, -0.26481685,  0.10215151, -0.32428357,  0.806316  ,
                     -0.07056033,  0.18556595,  0.21711856, -0.07236937,  0.27615574,
                     -0.5624484 , -0.26676714,  0.24473195,  0.03360732, -1.302939  ,
                     -1.089993  ,  1.9513313 ,  0.69385827,  2.5871234 , -1.0547583 ,
                     -0.12936595, -1.1408432 , -0.5611109 ,  0.19464505,  0.09307753,
                     -0.8203181 ,  1.1115559 , -1.2764733 , -1.0880976 , -0.11141836,
                     -0.4386923 , -0.32323474,  0.6297172 ,  1.5087334 , -0.6256103 ,
                     -0.4687212 , -0

In [ ]:
# from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.policy.policy import Policy, PolicySpec
# class DebugCallbacks(DefaultCallbacks):
#     def on_learn_on_batch(self, *, policy, train_batch, result, **kwargs):
#         print("Observations:", train_batch["obs"])
 
train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()
 
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hierarch_env", env_creator)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id.startswith('worker'):
        return "worker_policy"
    else:
        return "manager_policy"
 

worker_policy_spec = PolicySpec(
    observation_space=env.observation_space['worker'],
    action_space=env.action_space['worker'],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 
worker_config =   {"hmax": 50000,
        "initial_amount": 1000000,
        "trading_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  }
manager_config = {
    "n_stocks": stock_dimension,
}
 
trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5},local_dir="/Users/floriankockler/rayresults/training"),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),
 
    algorithm=a2c.A2C,
    config={
        "multiagent": {
            "policies": {
                "worker_policy": worker_policy_spec,
                "manager_policy": manager_policy_spec,
            },
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env": "hierarch_env",
        "env_config": {
 
        "worker_config" : worker_config,
        "manager_config": manager_config
        },
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()
 

In [3]:
# from ray.rllib.agents.callbacks import DefaultCallbacks

# class DebugCallbacks(DefaultCallbacks):
#     def on_learn_on_batch(self, *, policy, train_batch, result, **kwargs):
#         print("Observations:", train_batch["obs"])

train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment

register_env("hierarch_env", env_creator)

worker_config =   {"hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  }
manager_config = {
    "n_stocks": stock_dimension, 
}

trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5},local_dir="/Users/floriankockler/rayresults/training"),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),

    algorithm=a2c.A2C,
    config={
        "env": "hierarch_env",
        "env_config": {

        "worker_config" : worker_config,
        "manager_config": manager_config
        },
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},
    },
)
result = trainer.fit()

2023-08-31 20:49:33,471	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (13 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=3181) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=3181) 2023-08-31 20:49:38,283	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=3181) 2023-08-31 20:49:38,283	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=3181) 2023-08-31 20:49:38,283	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=3185) DeprecationWarning: `Di

(RolloutWorker pid=3186) _initiate_state: {'manager': array([66])}
(RolloutWorker pid=3186) self.state -manager init {'manager': array([66])}


(RolloutWorker pid=3185) 2023-08-31 20:49:43,483	WARNING env.py:298 -- Your MultiAgentEnv <HRL instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(RolloutWorker pid=3185) 2023-08-31 20:49:43,505	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=3185) 2023-08-31 20:49:43,506	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.tf_modelv2.TFModelV2` has been deprecated. Use `ray.rllib.core.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=3185) 2023-08-31 20:49:43,509	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.misc.normc_initializer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=3185) 2023-08-31 20:49:4

(RolloutWorker pid=3188) _initiate_state: {'manager': array([66])} [repeated 2x across cluster]
(RolloutWorker pid=3188) self.state -manager init {'manager': array([66])} [repeated 2x across cluster]


(RolloutWorker pid=3188) 2023-08-31 20:49:49,253	WARNING deprecation.py:50 -- DeprecationWarning: `compute_gradients` has been deprecated. This will raise an error in the future!
2023-08-31 20:49:49,924	ERROR tune_controller.py:911 -- Trial task failed for trial AIRA2C_1ffb1_00000
Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/_private/worker.py", line 2493, in get
    raise value.as_instanceof_cau

TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = RLTrainer.restore("/Users/floriankockler/rayresults/training/AIRA2C_2023-08-31_20-49-33")`.
To start a new run that will retry on training failures, set `air.RunConfig(failure_config=air.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.

_initiate_state: [nan]
self.state -manager init [nan]


AssertionError: Dict space element is not an instance of Space: key='dtype', space=<class 'numpy.float32'>

In [3]:
import ray
from ray.air.config import RunConfig, ScalingConfig
from ray.train.rl import RLTrainer
from ray.rllib.algorithms.bc.bc import BC
from ray.rllib.algorithms import a2c
from ray.tune.registry import register_env


def env_creator(env_config):
    return StockTradingEnvAdvanced(env_config)  # Assuming this is your environment

register_env("stock_trading_env", env_creator)

train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()

trainer = RLTrainer(
    run_config=RunConfig(stop={"training_iteration": 5}),
    scaling_config=ScalingConfig(
        num_workers=2,
        use_gpu=False,
    ),

    algorithm=a2c.A2C,
    config={
        "env": "stock_trading_env",
        "env_config": {
                       "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
        },
        "framework": "tf",
        "evaluation_num_workers": 1,
        "evaluation_interval": 1,
        "evaluation_config": {"input": "sampler"},

    },
)
result = trainer.fit()

2023-08-30 23:29:52,643	WARNING worker.py:2006 -- Warning: The actor AIRRLTrainer is very large (13 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=17038) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(AIRA2C pid=17038) 2023-08-30 23:29:58,085	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(AIRA2C pid=17038) 2023-08-30 23:29:58,085	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(AIRA2C pid=17038) 2023-08-30 23:29:58,086	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=17045) DeprecationWarning

(RolloutWorker pid=17055) day: 6304, episode: 3
(RolloutWorker pid=17055) begin_total_asset: 1000000.00
(RolloutWorker pid=17055) end_total_asset: 1407322.00
(RolloutWorker pid=17055) total_reward: 407322.00
(RolloutWorker pid=17055) total_cost: -9400030.60
(RolloutWorker pid=17055) total_trades: 44480
(RolloutWorker pid=17055) Sharpe: 0.170
(RolloutWorker pid=17055) Annual Return: 2.00%
(RolloutWorker pid=17055) =================================


(RolloutWorker pid=17055) 2023-08-30 23:31:00,278	WARNING env_runner_v2.py:155 -- More than 6305 observations in 6305 env steps for episode 89488672961613736 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.


(RolloutWorker pid=17055) day: 6304, episode: 4
(RolloutWorker pid=17055) begin_total_asset: 1000000.00
(RolloutWorker pid=17055) end_total_asset: 395646.19
(RolloutWorker pid=17055) total_reward: -604353.81
(RolloutWorker pid=17055) total_cost: -5735066.39
(RolloutWorker pid=17055) total_trades: 41228
(RolloutWorker pid=17055) Sharpe: -0.058
(RolloutWorker pid=17055) Annual Return: -5.23%
(RolloutWorker pid=17055) =================================
(RolloutWorker pid=17055) day: 6304, episode: 5
(RolloutWorker pid=17055) begin_total_asset: 1000000.00
(RolloutWorker pid=17055) end_total_asset: 283767.16
(RolloutWorker pid=17055) total_reward: -716232.84
(RolloutWorker pid=17055) total_cost: -6171214.87
(RolloutWorker pid=17055) total_trades: 43055
(RolloutWorker pid=17055) Sharpe: -0.130
(RolloutWorker pid=17055) Annual Return: -7.03%
(RolloutWorker pid=17055) =================================


In [ ]:
# worker_config =   {"hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  }
# manager_config = {
#     "n_stocks": stock_dimension, 
# }

# config = {
#     "worker_config": worker_config,
#     "manager_config": manager_config
# }

# env = HierarchicalTradingEnv(config)
# obs = env.reset()
# for _ in range(30):
#     action = {
#         "manager": env.action_space["manager"].sample(),
#         "worker": env.action_space["worker"].sample()
#     }
#     new_obs, reward, done, truncated, info = env.step(action)
#     print(new_obs, reward, done)



In [ ]:
# train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()
# worker_config =   {"hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  }
# manager_config = {
#     "n_stocks": stock_dimension, 
# }

# config = {
#     "worker_config": worker_config,
#     "manager_config": manager_config
# }


# env = HierarchicalTradingEnv(config)
# obs = env.reset()
# for _ in range(30):
#     action = {
#         "manager": env.action_space["manager"].sample(),
#         "worker": env.action_space["worker"].sample()
#     }
#     new_obs, reward, done, truncated, info = env.step(action)
#     print(new_obs, reward, done)


In [ ]:
# import numpy as np
# from env.multi_agent.worker import Worker
# from env.multi_agent.manager import Manager

# train_df, validate_df, test_df, stock_dimension, state_space, indicators = pharma_basket()

# worker_config =   {"hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  }
# manager_config = {
#     "n_stocks": stock_dimension, 
# }

# env_config =   {"hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  }


# # Initialize the environment
# env = Manager(manager_config, worker_config)

# # Define the number of episodes and steps per episode
# num_episodes = 10
# steps_per_episode = 100

# for episode in range(num_episodes):
#     obs = env.reset()
#     done = False
#     total_reward = 0
#     for step in range(steps_per_episode):
#         if done:
#             break
#         # Take a random action from the action space
#         action = np.random.uniform(env.action_space.low, env.action_space.high)
#         obs, reward, done, trunctudated, info = env.step(action)
#         total_reward += reward
#     print(f"Episode {episode + 1}: Total Reward = {total_reward}")

# print("Random testing completed!")

In [ ]:
df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df
df = forward_fill_missing_stock_data(df)
from ray.tune.registry import register_env

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)


stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension


config = {
    "log_level": 'ERROR',
    "env": StockTradingEnvAdvanced,
    "env_config": {
        "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    },
    "evaluation_config": {
        "env_config": {
            "df": validate_df,
        }
    },
    "evaluation_num_episodes": 10
}

def env_creator(env_config=config):
    return StockTradingEnvAdvanced(env_config)  # Assuming this is your environment

register_env("stock_trading_env", env_creator)

config = A2CConfig()
config = config.training(lr=0.01, grad_clip=30.0)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=2)  
config.environment(disable_env_checking=True)
# config = config.environment("CartPole-v1", config=)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build()  
algo.train()

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'torch_compile_learner': False, 'torch_compile_learner_what_to_compile': <TorchCompileWhatToCompile.FORWARD_TRAIN: 'forward_train'>, 'torch_compile_learner_dynamo_backend': 'aot_eager', 'torch_compile_learner_dynamo_mode': None, 'torch_compile_worker': False, 'torch_compile_worker_dynam

(pid=79577) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-29 23:45:59,810	ERROR actor_manager.py:500 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=79577, ip=127.0.0.1, actor_id=a87a18720059d76f18e5a22301000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x169777ac0>)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 470, in __init__
    self.policy_dict, self.is_policy_to_train = self.config.get_multi_agent_setup(
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm_config.py", line 2932, in get_multi_agent_setup
    raise ValueError(
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces recei

ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!

In [ ]:
#RL Trainer

from ray.tune.registry import register_env
from ray.rllib.algorithms import a2c
from ray.air import ScalingConfig
from ray import air, tune
from ray.air import RunConfig
from ray.train.trainer import BaseTrainer
from ray.train.rl.rl_trainer import RLTrainer
from ray.rllib.algorithms.callbacks import MemoryTrackingCallbacks
from ray.rllib.algorithms import a2c

env_config= {
         "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    }

def env_creator(env_config=env_config):
    return StockTradingEnvAdvanced(env_config)  # Assuming this is your environment

register_env("stock_trading_env", env_creator)

env_config= {
         "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    }
# config = A2CConfig().training(gamma=0.9, lr=0.01).environment(env="stock_trading_env", env_config=env_config).resources(num_gpus=0).rollouts(num_rollout_workers=4).callbacks(MemoryTrackingCallbacks)



df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df
df = forward_fill_missing_stock_data(df)

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)


stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension

def env_creator(env_config=env_config):
    return StockTradingEnvAdvanced(env_config)  # Assuming this is your environment

register_env("stock_trading_env", env_creator)

static_config = {
    "log_level": 'ERROR',
    "env": StockTradingEnvAdvanced,
    "env_config": {
        "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    },
    "evaluation_config": {
        "env_config": {
            "df": validate_df,
        }
    },
    "evaluation_num_episodes": 10
}

config = A2CConfig(config=static_config)
config = config.training(lr=0.01, grad_clip=30.0)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=2)  
config.environment(disable_env_checking=True)
# config = config.environment("CartPole-v1", config=)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build()  
algo.train()

NameError: name 'state_space' is not defined

In [ ]:
from ray.tune.registry import register_env
from ray.rllib.algorithms import a2c
from ray.air import ScalingConfig
from ray import air, tune
from ray.air import RunConfig


def env_creator(env_config):
    return StockTradingEnvAdvanced(env_config)  # Assuming this is your environment

register_env("stock_trading_env", env_creator)

param_space = {
    "scaling_config": ScalingConfig(
        num_workers=tune.grid_search([2, 4]),
        resources_per_worker={
            "CPU": tune.grid_search([1, 2]),
        },
    ),
    "params": {
           "lr": tune.loguniform(1e-4, 1e-2),  # Learning rate
            "gamma": tune.uniform(0.9, 0.99),   # Discount factor
            "entropy_coeff": tune.loguniform(1e-4, 1e-1), #This coefficient determines the importance of the entropy term in the loss function, promoting exploration.
            "vf_loss_coeff": tune.uniform(0.1, 1.0), #scales the value function loss in the combined loss function.
            # "lr_schedule": tune.choice(["constant", "linear_decay"]), #Some models benefit from adjusting the learning rate during training.
            "lambda": tune.uniform(0.9, 1.0), #Generalized Advantage Estimation (GAE) parameter can be tuned to balance bias and variance in the advantage estimation
            "num_steps": tune.choice([5, 10, 20, 50]), #number of steps to roll out for each batch can impact training dynamics
            "num_timesteps": tune.quniform(50000, 1000000, 10000)
    },
}

env_config= {
         "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    }

trainer = a2c.A2C(env="stock_trading_env", config=env_config)  # Include any default config you need

tuner = tune.Tuner(trainable=trainer, param_space=param_space, run_config=RunConfig(name="my_tune_run"))

results = tuner.fit()

# config = {
#     "env": "StockTradingEnvAdvanced-v1",
#     "num_gpus": 0,
#     "num_workers": 1,
#     "lr": tune.grid_search([0.01, 0.001, 0.0001]),
#     "env_config": {
#          "hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  
#     }
# }


# config = {
# "env_config" : {
#         "hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": train_df,  

# }
#     }




2023-08-29 23:34:22,597	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property hmax not supported.
2023-08-29 23:34:22,600	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='stock_trading_env', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('stock_trading_env').build()` instead. This will raise an error in the future!
(pid=78241) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=78241) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=78241, ip=127.0.0.1, actor_id=f08fe51bfa99514d874d72f101000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x178f5fa60>)
(RolloutWorker pid=78241)   File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 397, in __init__
(RolloutWo

KeyError: 'df'

In [ ]:
from ray import tune
from ray.tune import Callback
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import MedianStoppingRule, HyperBandScheduler, AsyncHyperBandScheduler
import time
from ray.tune import CLIReporter
from models.callbacks import RayTuneCustomTradesLoggingCallback
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.search import ConcurrencyLimiter

class ElapsedTimeCallback(Callback):
    def on_trial_start(self, iteration, trials, trial):
        # Store the start time for this trial
        self.trial_start_time = time.time()

    def on_trial_complete(self, iteration, trials, trial, **info):
        # Calculate elapsed time for this trial
        elapsed_time = time.time() - self.trial_start_time
        print(f"Trial {trial.trial_id} completed. Time taken: {elapsed_time:.2f} seconds.")



callbacks_list = [ElapsedTimeCallback(), RayTuneCustomTradesLoggingCallback(log_freq=5000)]


# Define the search space
search_space = {
    "lr": tune.loguniform(1e-4, 1e-2),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),   # Discount factor
    "entropy_coeff": tune.loguniform(1e-4, 1e-1), #This coefficient determines the importance of the entropy term in the loss function, promoting exploration.
    "vf_loss_coeff": tune.uniform(0.1, 1.0), #scales the value function loss in the combined loss function.
    # "lr_schedule": tune.choice(["constant", "linear_decay"]), #Some models benefit from adjusting the learning rate during training.
    "lambda": tune.uniform(0.9, 1.0), #Generalized Advantage Estimation (GAE) parameter can be tuned to balance bias and variance in the advantage estimation
    "num_steps": tune.choice([5, 10, 20, 50]), #number of steps to roll out for each batch can impact training dynamics
    "num_timesteps": tune.quniform(50000, 1000000, 10000)
}



hyperopt_search = HyperOptSearch(search_space, metric="mean_reward", mode="max")
hyperband_scheduler = HyperBandScheduler(metric="mean_reward", mode="max")
async_hyperband = AsyncHyperBandScheduler(grace_period=5, max_t=100,metric="mean_reward", mode="max")

reporter = CLIReporter(max_progress_rows=10)


def ray_tuner():
    start_time = time.time()  # Start the timer
    # early_stopping = MedianStoppingRule(grace_period=5, hard_stop=False)
    config = {
    "log_level": 'ERROR',
    "env": StockTradingEnvAdvanced,
    "env_config": {
        "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity":1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    },
    "evaluation_config": {
        "env_config": {
            "df": validate_df,
        }
    },
    "evaluation_num_episodes": 10,  # Number of episodes to evaluate on
    # Add other hyperparameters and their search spaces as needed

}

    analysis = tune.run(
       "A2C",
    stop={"training_iteration": 155},
    scheduler=async_hyperband,
    search_alg=hyperopt_search,
    # callbacks=callbacks_list,
    progress_reporter=reporter,
    num_samples=10,
    # resources_per_trial={"cpu": 4, "gpu": 0},
    config=config,
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "mean_reward",
        "checkpoint_at_end": "True",
    },
    local_dir="/Users/floriankockler/rayresults/tune_StockEnv/",
    )
    # Get the best trial object based on a specific metric and mode
    best_trial = analysis.get_best_trial("mean_reward", mode="max", scope="all")

    # Get the configuration and last result of the best trial
    best_trial_config = best_trial.config
    best_trial_result = best_trial.last_result

    print("Best Trial Config:", best_trial_config)
    print("Best Trial Result:", best_trial_result)

    # Retrieve all trial data as a DataFrame
    df = analysis.dataframe()

    return df



# ray_tuner()

async_hyperband = AsyncHyperBandScheduler(
    metric="mean_reward", 
    mode="max",
    max_t=100,
    grace_period=5
)

static_config = {
    "log_level": 'ERROR',
    "env": StockTradingEnvAdvanced,
    "env_config": {
        "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df,  
    },
    "evaluation_config": {
        "env_config": {
            "df": validate_df,
        }
    },
    "evaluation_num_episodes": 10
}

search_space = {
    "lr": tune.loguniform(0.00001, 0.0001),
    "gamma": tune.uniform(0.95, 0.9999),
    "entropy_coeff": tune.loguniform(0.01, 0.1),
    "vf_loss_coeff": tune.uniform(0.1, 0.5),
    "lambda": tune.uniform(0.9, 1.0),
    "num_steps": tune.choice([5, 10, 20, 50]),
    "num_timesteps": tune.quniform(50000, 1000000, 10000)
}

# Merge the two configurations
config = {**static_config, **search_space}
# config = {**static_config}

tune.run(
    "A2C",
    stop={"training_iteration": 150},
    config=config,
    num_samples=10,


    callbacks=callbacks_list,
    # scheduler=async_hyperband, 
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "mean_reward",
        "checkpoint_at_end": "True",
    },
    progress_reporter=reporter,
    local_dir="/Users/floriankockler/rayresults/tune_StockEnv2/"
)

# pbt = PopulationBasedTraining(
#     time_attr="training_iteration",
#     metric="mean_reward",
#     mode="max",
#     perturbation_interval=10,  # every N time_attr units, "perturb" the parameters for specified trials.
#     hyperparam_mutations={
#     "lr": tune.loguniform(0.00001, 0.0001),
#     "gamma": tune.uniform(0.95, 0.9999),
#     "entropy_coeff": tune.loguniform(0.01, 0.1),
#     "vf_loss_coeff": tune.uniform(0.1, 0.5),
#     "lambda": tune.uniform(0.9, 1.0),
#     "num_steps": tune.choice([5, 10, 20, 50]),
#     "num_timesteps": tune.quniform(50000, 1000000, 10000)
#     }
# )

# tune.run(
#     "A2C",
#     stop={"training_iteration": 150},
#     config=config,
#     scheduler=pbt,  # Using PBT scheduler here
#     num_samples=10,
#     callbacks=callbacks_list,
#     checkpoint_config={
#         "num_to_keep": 3,
#         "checkpoint_score_attribute": "mean_reward",
#         "checkpoint_at_end": "True",
#     },
#     progress_reporter=reporter,
#     local_dir="/Users/floriankockler/rayresults/tune/"
# )

2023-08-29 23:35:05,167	WARNING hyperopt_search.py:194 -- You passed a `space` parameter to <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> that contained unresolved search space definitions. <class 'ray.tune.search.hyperopt.hyperopt_search.HyperOptSearch'> should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.
2023-08-29 23:35:05,181	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-08-29 23:35:05,183	WARNING syncer.py:260 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage prov

== Status ==
Current time: 2023-08-29 23:35:11 (running for 00:00:05.35)
Using FIFO scheduling algorithm.
Logical resource usage: 0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tune_StockEnv2/A2C
Number of trials: 10/10 (10 PENDING)
+-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------+
| Trial name                              | status   | loc   |   entropy_coeff |    gamma |   lambda |          lr |   num_steps |   num_timesteps |   vf_loss_coeff |
|-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------|
| A2C_StockTradingEnvAdvanced_eb3e6_00000 | PENDING  |       |       0.0134189 | 0.994331 | 0.965835 | 3.46622e-05 |          50 |          480000 |        0.228266 |
| A2C_StockTradingEnvAdvanced_eb3e6_00001 | PENDING  |       |       0.0131788 

(pid=78446) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2023-08-29 23:35:21 (running for 00:00:15.49)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tune_StockEnv2/A2C
Number of trials: 10/10 (10 PENDING)
+-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------+
| Trial name                              | status   | loc   |   entropy_coeff |    gamma |   lambda |          lr |   num_steps |   num_timesteps |   vf_loss_coeff |
|-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------|
| A2C_StockTradingEnvAdvanced_eb3e6_00000 | PENDING  |       |       0.0134189 | 0.994331 | 0.965835 | 3.46622e-05 |          50 |          480000 |        0.228266 |
| A2C_StockTradingEnvAdvanced_eb3e6_00001 | PENDING  |       |       0.013178

(A2C pid=78446) 2023-08-29 23:35:21,713	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=78446) 2023-08-29 23:35:21,715	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=78446) 2023-08-29 23:35:21,715	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_steps not supported.
(A2C pid=78446) 2023-08-29 23:35:21,715	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!


== Status ==
Current time: 2023-08-29 23:35:26 (running for 00:00:20.50)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tune_StockEnv2/A2C
Number of trials: 10/10 (10 PENDING)
+-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------+
| Trial name                              | status   | loc   |   entropy_coeff |    gamma |   lambda |          lr |   num_steps |   num_timesteps |   vf_loss_coeff |
|-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------|
| A2C_StockTradingEnvAdvanced_eb3e6_00000 | PENDING  |       |       0.0134189 | 0.994331 | 0.965835 | 3.46622e-05 |          50 |          480000 |        0.228266 |
| A2C_StockTradingEnvAdvanced_eb3e6_00001 | PENDING  |       |       0.013178

(pid=78494) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=78493) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2023-08-29 23:35:36 (running for 00:00:30.53)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tune_StockEnv2/A2C
Number of trials: 10/10 (10 PENDING)
+-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------+
| Trial name                              | status   | loc   |   entropy_coeff |    gamma |   lambda |          lr |   num_steps |   num_timesteps |   vf_loss_coeff |
|-----------------------------------------+----------+-------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------|
| A2C_StockTradingEnvAdvanced_eb3e6_00000 | PENDING  |       |       0.0134189 | 0.994331 | 0.965835 | 3.46622e-05 |          50 |          480000 |        0.228266 |
| A2C_StockTradingEnvAdvanced_eb3e6_00001 | PENDING  |       |       0.013178

(RolloutWorker pid=78494) /Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/gymnasium/wrappers/compatibility.py:60: DeprecationWarning: WARN: The `gymnasium.make(..., apply_api_compatibility=...)` parameter is deprecated and will be removed in v1.0. Instead use `gymnasium.make('GymV21Environment-v0', env_name=...)` or `from shimmy import GymV21CompatibilityV0`
(RolloutWorker pid=78494)   logger.deprecation(
(A2C pid=78446) Trainable.setup took 17.778 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


== Status ==
Current time: 2023-08-29 23:35:41 (running for 00:00:35.63)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /Users/floriankockler/rayresults/tune_StockEnv2/A2C
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-----------------------------------------+----------+-----------------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------+
| Trial name                              | status   | loc             |   entropy_coeff |    gamma |   lambda |          lr |   num_steps |   num_timesteps |   vf_loss_coeff |
|-----------------------------------------+----------+-----------------+-----------------+----------+----------+-------------+-------------+-----------------+-----------------|
| A2C_StockTradingEnvAdvanced_eb3e6_00000 | RUNNING  | 127.0.0.1:78446 |       0.0134189 | 0.994331 | 0.965835 | 3.46622e-05 |          50 |          480000 |        0.228266 |
| A2C_StockTradingEnvAdvanc

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm

my_best_a2c = Algorithm.from_checkpoint("/Users/floriankockler/rayresults/tune_StockEnv2/A2C/A2C_StockTradingEnvAdvanced_d9e46_00001_1_entropy_coeff=0.0143,gamma=0.9588,lambda=0.9862,lr=0.0001,num_steps=5,num_timesteps=1600_2023-08-29_01-44-38/checkpoint_000150")

2023-08-29 20:56:03,028	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property num_steps not supported.
2023-08-29 20:56:03,046	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
2023-08-29 20:56:03,046	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(pid=52402) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=52402) /Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/gymnasium/wrappers/compatibility.py:60: DeprecationWarning: WARN: The `gymnasium.make(..., apply_api_compatibility=...)` parameter is deprecated and will be removed in v1.0. Instead use `gymnasium.make('GymV21Environment-v0', env_name=...)` or `from shimmy imp

In [ ]:
my_best_a2c.save("./saved_models/")

'./saved_models/checkpoint_000000'

In [ ]:
len(test_df[test_df["tic"] == "HUM.US"]["date"].unique())

1917

In [ ]:
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
env_test_config = {
        "hmax": 50000, 
        "initial_amount": 1000000, 
        "trading_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": indicators,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": test_df,  
    }
env_test = StockTradingEnvAdvanced(env_test_config)

# Assuming you've loaded your RLlib model as `my_new_ppo`

policy = my_best_a2c.get_policy(policy_id="default_policy")

def DRL_prediction_flo_rllib(policy, environment):
    state = environment.reset()
    done = False
    account_memory = []
    actions_memory = []
    return_memory = []
    portfolio_memory = []

    for i in range(len(environment.df.index.unique())):
        action, _, _ = policy.compute_single_action(state)  # Adjusted this line
        state, reward, done, info = environment.step(action)
        



        # Check if it's the second last iteration
        if i == (len(environment.df.index.unique()) - 2):
            print("Consideration is met")
            account_memory = environment.save_asset_memory()
            actions_memory = environment.save_action_memory()
            return_memory = environment.save_return_memory()
            portfolio_memory = environment.save_portfolio_memory()

        if done:
            print("hit end!")
            break

    return account_memory, actions_memory,return_memory, portfolio_memory

# Assuming `env_test` is already set up to use test data
df_account_memory, df_actions, df_daily_return, df_df_daily_return = DRL_prediction_flo_rllib(policy=policy, environment=env_test)



Consideration is met
day: 1917, episode: 1
begin_total_asset: 1000000.00
end_total_asset: 4135005.25
total_reward: 3135005.25
total_cost: -265574.00
total_trades: 1980
Sharpe: 0.989
Annual Return: 31.03%
hit end!


In [ ]:
df_account_memory

,date,account_value
0,2016-01-04,1.000000e+06
1,2016-01-05,1.004927e+06
2,2016-01-06,9.931112e+05
3,2016-01-07,9.689408e+05
4,2016-01-08,9.525500e+05
...,...,...
1913,2023-08-10,3.604494e+06
1914,2023-08-11,3.639864e+06
1915,2023-08-14,3.682657e+06
1916,2023-08-15,3.741644e+06


In [ ]:
df_actions

,ABT.US,AMGN.US,BDX.US,BMY.US,HUM.US,JNJ.US,LLY.US,MDT.US,MRK.US,PFE.US,SYK.US,TMO.US,UNH.US
date,,,,,,,,,,,,,
2016-01-04,8073,2378,2306,0,0,1208,0,0,0,0,0,0,0
2016-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-01-06,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-01-07,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-01-08,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-09,0,0,0,0,0,0,0,0,0,0,0,0,0
2023-08-10,-1320,0,497,3,0,0,0,0,0,0,0,0,0
2023-08-11,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_daily_return

,date,daily_return
0,2016-01-04,0.000000
1,2016-01-05,-0.000024
2,2016-01-06,-0.008765
3,2016-01-07,-0.020430
4,2016-01-08,-0.016297
...,...,...
1913,2023-08-10,-0.003228
1914,2023-08-11,0.009906
1915,2023-08-14,0.011969
1916,2023-08-15,0.016075


In [ ]:
df_df_daily_return

,ABT.US,AMGN.US,BDX.US,BMY.US,HUM.US,JNJ.US,LLY.US,MDT.US,MRK.US,PFE.US,SYK.US,TMO.US,UNH.US,date
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-04
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-05
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-06
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-07
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-10
1914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-11
1915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-14
1916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-15


In [ ]:
# from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
# env_test_config = {
#         "hmax": 50000, 
#         "initial_amount": 1000000, 
#         "trading_cost_pct": 0.001, 
#         "state_space": state_space, 
#         "stock_dim": stock_dimension, 
#         "tech_indicator_list": indicators,
#         "action_space": stock_dimension, 
#         "reward_scaling": 1e-4,
#         "print_verbosity": 1,
#         "num_stock_shares": np.zeros(stock_dimension).tolist(),
#         "df": test_df,  
#     }
# env_test = StockTradingEnvAdvanced(env_test_config)

# # Assuming you've loaded your RLlib model as `my_new_ppo`

# policy = my_best_a2c.get_policy(policy_id="default_policy")

# def DRL_prediction_flo_rllib(policy, environment):
#     state = environment.reset()
#     done = False
#     account_memory = []
#     actions_memory = []
#     return_memory = []
#     portfolio_memory = []

#     while not done:
#         action, _, _ = policy.compute_single_action(state)  # Adjusted this line
#         state, reward, done, info = environment.step(action)
        
#         # Assuming the environment has these methods to store memories
#         # df_actions.append(environment.save_action_memory())
#         # df_return_memory.append(environment.save_return_memory())
#         # df_account_value.append(environment.save_asset_memory())

#         # Check if it's the second last iteration
#         if (len(environment.df.index.unique()) -  2):
#             print("Consideration is met")
#             account_memory = environment.save_asset_memory()
#             actions_memory = environment.save_action_memory()
#             return_memory = environment.save_return_memory()
#             portfolio_memory = environment.save_portfolio_memory()


#         if done:
#             print("hit end!")
#             break

#     return account_memory[0], actions_memory[0],return_memory[0], portfolio_memory[0]

# # Assuming `env_test` is already set up to use test data
# df_account_memory, df_actions, df_daily_return, portfolio_memory_df  = DRL_prediction_flo_rllib(policy=policy, environment=env_test)


In [ ]:
# from ray.rllib.utils import check_env
# from env.multi_agent.manager import PortfolioManagerEnv
# from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
# from env.env_ST_advanced_raylib import StockTradingEnvAdvanced

# df = pd.read_pickle("pharma.pkl")
# df = df[df['tic'] != 'NVO.US']
# df = df[df['tic'] != 'DHR.US']
# df

# train_start = "1986-01-02"
# train_end = "2010-12-31"
# validate_start = "2011-01-01"
# validate_end = "2015-12-31"
# test_start = "2016-01-01"
# test_end = "2023-08-17"


# indicators = [ 'avgvol_50',
#        'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
#        'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
#        'macd', 'atr_14',"volume"]

# def data_split(df, start, end, target_date_col="date"):
#     data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
#     data = data.sort_values([target_date_col, "tic"], ignore_index=True)
#     data.index = data[target_date_col].factorize()[0]
#     return data
# train_df = data_split(df, train_start, train_end)
# validate_df = data_split(df, validate_start, validate_end)
# test_df = data_split(df, test_start, test_end)

# stock_dimension = len(train_df.tic.unique())
# state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension
# manager_config = {
#     "n_stocks": stock_dimension
# }
# worker_config = {
#     "n_stocks": stock_dimension,
#     "hmax": 50000,
#     "initial_amount": 1000000,
#     "trading_cost_pct": 0.001,
#     "state_space": state_space,
#     "stock_dim": stock_dimension,
#     "tech_indicator_list": indicators,
#     "action_space": stock_dimension,
#     "reward_scaling": 1e-4,
#     "print_verbosity": 1,
#     "num_stock_shares": np.zeros(stock_dimension).tolist(),
#     "df": train_df
# }
# # Instantiate your environments
# manager_env_instance = PortfolioManagerEnv(manager_config=manager_config, worker_config=worker_config)
# worker_env_instance = manager_env_instance.worker_env

# # Check the manager environment
# check_env(manager_env_instance)

# # Check the worker environment
# check_env(worker_env_instance)

In [ ]:
from ray.rllib.algorithms.a2c import A2C
from env.multi_agent.manager import PortfolioManagerEnv
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from ray.rllib.policy.policy import Policy, PolicySpec
df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)

stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension
def policy_mapping_fn(agent_id):
    return agent_id  # simply return the agent_id ("manager" or "worker")

# Define manager and worker configurations
manager_config = {
    "n_stocks": stock_dimension
}
worker_config = {
    "n_stocks": stock_dimension,
    "hmax": 50000,
    "initial_amount": 1000000,
    "trading_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicators,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity": 1,
    "num_stock_shares": np.zeros(stock_dimension).tolist(),
    "df": train_df
}

# Instantiate the environments to derive shapes
manager_env = PortfolioManagerEnv(manager_config=manager_config, worker_config=worker_config)

# Extract the shapes
manager_observation_shape = manager_env.observation_space.shape
worker_observation_shape = manager_env.worker_env.observation_space.shape

# Define the a2c_config using the derived shapes

def policy_mapping_function(agent_id, episode=None, worker=None, **kwargs):
    return agent_id 


a2c_config = {
    "lr": 0.0001,
    "gamma": 0.99,
    "use_gae": True,
    "lambda": 0.97,
    "num_workers": 1,
    "train_batch_size": 5000,
    "model": {
        "fcnet_hiddens": [256, 256],  # Size of the two hidden layers
        "fcnet_activation": "tanh",  # Activation function
    },
    "multiagent": {
        "policies": {
            "manager": (None, manager_env.observation_space, manager_env.action_space, {}),
            "worker": (None, manager_env.worker_env.observation_space, manager_env.worker_env.action_space, {}),
        },
        "policy_mapping_fn": policy_mapping_function
    },
    "env_config": {
        "manager_config": manager_config,
        "worker_config": worker_config
    }
}

# Initialize the PPO algorithm with the HierarchicalTradingEnv
algo = A2C(env=HierarchicalTradingEnv, config=a2c_config)

# Training loop
while True:
    result = algo.train()
    print(f"Mean Reward: {result['episode_reward_mean']}")

2023-08-30 16:23:11,141	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
2023-08-30 16:23:11,142	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
2023-08-30 16:23:11,143	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>').build()` instead. This will raise an error in the future!
(pid=33881) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=33881) 2023-08-30 16:23:18,376	WARNING env.py:298 -- Your MultiAgentEnv <HierarchicalTradingEnv instance> does

RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=33881, ip=127.0.0.1, actor_id=e4c2252839a06156810e502801000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x15785fa60>)
ValueError: The two structures don't have the same nested structure.

First structure: type=tuple str=(array([ 1.0000000e+06,  8.2849997e-01,  0.0000000e+00,  6.6008695e+06,
        8.0900002e-01,  7.5940001e-01,  7.2000003e-01,  6.8720001e-01,
        7.8160000e-01,  6.3639500e+01,  1.0000000e-03,  2.6000000e-03,
        3.2121200e-02,  3.5865200e+01,  3.8922901e+01,  1.7600000e-02,
        1.4414001e+00,  6.4513130e+06,  1.9200000e-01,  0.0000000e+00,
        2.7120000e+06,  2.0250000e-01,  1.7739999e-01,  1.4830001e-01,
        1.2500000e-01,  1.8660000e-01,  5.1070801e+01,  3.0000001e-04,
        1.1000000e-03,  1.6757365e-02,  1.3611200e+01,  3.7022499e+01,
        7.7999998e-03,  5.7950002e-01,  8.8800000e+05,  2.1572001e+00,
        0.0000000e+00,  1.4716704e+06,  2.1882

In [ ]:
from ray.rllib.algorithms.a2c import A2C
from env.multi_agent.manager import PortfolioManagerEnv
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from ray.rllib.policy.policy import Policy, PolicySpec
df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)

stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension

# HierarchicalTradingEnv configuration
env_config = {
    "manager_config": {
        "n_stocks": stock_dimension
    },
    "worker_config": {
        "hmax": 50000,
        "initial_amount": 1000000,
        "trading_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df
    }
}


a2c_config = {
    "lr": 0.0001,
    "gamma": 0.99,
    "use_gae": True,
    "lambda": 0.97,
    "num_workers": 1,
    "train_batch_size": 5000,
    "model": {
        "fcnet_hiddens": [256, 256],  # Size of the two hidden layers
        "fcnet_activation": "tanh",  # Activation function
    },
    "manager_config": {
            "n_stocks": stock_dimension
        },
    "worker_config": {
        "n_stocks": stock_dimension,
        "hmax": 50000,
        "initial_amount": 1000000,
        "trading_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df
    },
}
a2c_config2 = {
    "lr": 0.0001,
    "gamma": 0.99,
    "use_gae": True,
    "lambda": 0.97,
    "num_workers": 1,
    "train_batch_size": 5000,
    "model": {
        "fcnet_hiddens": [256, 256],  # Size of the two hidden layers
        "fcnet_activation": "tanh",  # Activation function
    },
    "env_config": {
    "manager_config": {
            "n_stocks": stock_dimension
        },
    "worker_config": {
        "n_stocks": stock_dimension,
        "hmax": 50000,
        "initial_amount": 1000000,
        "trading_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df
    },

    }
}

# Initialize the PPO algorithm with the HierarchicalTradingEnv
algo = A2C(env=HierarchicalTradingEnv, config=a2c_config2)

# Training loop
while True:
    result = algo.train()
    print(f"Mean Reward: {result['episode_reward_mean']}")

2023-08-30 16:25:09,692	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>').build()` instead. This will raise an error in the future!
(pid=34232) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=34232) 2023-08-30 16:25:16,584	WARNING env.py:298 -- Your MultiAgentEnv <HierarchicalTradingEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(RolloutWorker pid=34232) 2023-08-30 16:25:16,598	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in

RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=34232, ip=127.0.0.1, actor_id=d6b8344d5d48f1706ce79acf01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x15b9b7af0>)
ValueError: The two structures don't have the same nested structure.

First structure: type=ndarray str=[1000000. 1000000.       0.       0.       0.       0.       0.       0.
       0.       0.       0.       0.       0.       0.       0.       0.]

Second structure: type=OrderedDict str=OrderedDict([('manager', array([0.4107395 , 1.6530467 , 1.6854217 , 0.66274625, 1.5941068 ,
       3.6271145 , 0.14428025, 1.8825939 , 0.44741958, 0.44989774,
       0.42876312, 1.399072  , 2.9491625 , 0.7723294 , 0.59304744,
       0.23955488], dtype=float32)), ('worker', array([ 1.69525397e+00,  1.57627928e+00, -3.47433895e-01, -2.50792980e-01,
        1.16043854e+00,  4.60300803e-01,  1.15869999e+00, -4.93435025e-01,
       -3.39585274e-01,  3.01672488e-01, -1.20959592e+00, -1.87990451e+00

In [ ]:

df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)

stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension

config={

      "hmax": 50000, 
    "initial_amount": 1000000, 
    "trading_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": indicators,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":1,
    "num_stock_shares": np.zeros(stock_dimension).tolist(),
    "df": train_df,  


    }




In [ ]:
from __future__ import annotations
from env.env_components.reward import RewardFunction, RewardTradeBudgetPenalty, RewardTradeBudgetPenalty
from env.env_components.trading import TradingFunction, TradingOrdinary

from typing import List
from datetime import datetime
import gymnasium as gym
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gymnasium import spaces
from gymnasium.utils import seeding

from collections import deque, defaultdict
matplotlib.use("Agg")

class Spec(object):
    def __init__(self, id, max_episode_steps):
        self.id = id
        self.max_episode_steps = max_episode_steps


class StockTradingEnvAdvanced(gym.Env):

    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        env_config: None,
        day=0,
        previous_state=[],
        model_name="",
        mode="",
        iteration="",
        asset_reward_weight=0.5,
        sharpe_reward_weight=0.5,
        dynamic_weighting=True,
        max_position_weight=0.30,
        trade_ratio: int = 0.17,
        trade_penalty_factor=-10000,
        reward_scaling: float = 1e-3,
        hmax: int = 70000,
    ):
        if env_config is None:
            env_config = {}
        self.day = day
        self.df = env_config["df"]
        self.spec = Spec(id="StockTradingEnvAdvanced-v0", max_episode_steps=len(self.df.index.unique()) - 1)
        self.data = self.df.loc[self.day, :]
        self.tickers_list = list(self.df.tic.unique())
        self.number_of_trade_budget = int(
            len(self.df) * trade_ratio * len(self.tickers_list))

        self.trade_penalty_factor = trade_penalty_factor
        self.current_step_cost = 0

        self.stock_dim = env_config.get("stock_dim")
        self.hmax = env_config.get("hmax", hmax)
        self.num_stock_shares = env_config.get(
            "num_stock_shares")

        self.initial_amount = env_config.get(
            "initial_amount")  # get the initial cash
        self.trading_cost_pct = env_config.get(
            "trading_cost_pct")

        self.max_position_weight = max_position_weight

        self.reward_scaling = env_config["reward_scaling"]
        self.state_space = env_config.get("state_space")
        self.action_space = env_config.get("action_space")
        self.tech_indicator_list = env_config.get(
            "tech_indicator_list")
        # ? Action & Observation Space
        # initalize state
        self.action_space = spaces.Box(
            low=-1, high=1, shape=(self.action_space,))

        self.timestep_since_last_trade = 0
        self.trades = 0
        self.state = self._initiate_state_hstack()
        self.state_components = [
            self._get_cash_balance,
            self._get_stock_prices,
            self._get_stock_owned,
            self._get_tech_indicators,
            self._get_current_step_cost,
            self._get_timestep_since_last_trade,
            self._get_number_of_trade_budget,
            self._get_number_of_done_trades

        ]
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.calculate_state_shape(),), dtype=np.float32
        )
        self.terminal = False
        self.print_verbosity = env_config.get("print_verbosity")
        self.reward_function = RewardTradeBudgetPenalty(self)
        self.trading_function = TradingOrdinary(self)

        self.previous_state = previous_state
        self.model_name = model_name
        self.mode = mode
        self.iteration = iteration
        self.stocks_owned = {ticker: initial_holding for ticker, initial_holding in zip(
            self.tickers_list, self.num_stock_shares)}

        # ? initialize reward related stuff
        self.dynamic_weighting = dynamic_weighting
        self.asset_reward_weight = asset_reward_weight
        self.sharpe_reward_weight = sharpe_reward_weight
        self.portfolio_return_memory = [0]
        self.reward = 0
        # ? Track Trading behavior
        self.total_costs = 0
        self.episode = 0
        self.asset_memory = [
            self.initial_amount
            + np.sum(
                np.array(self.num_stock_shares)
                * np.array(self.state[1::(2 + len(self.tech_indicator_list))][:self.stock_dim])
            )
        ]
        self.portfolio_allocation_memory = []
        # ? Track Performance
        self.episode_end_date = []
        self.episode_start_date = []
        self.rewards_memory = []
        self.actions_memory = []
        self.state_memory = (
            []
        )  # we need sometimes to preserve the state in the middle of trading process
        self.date_memory = [self._get_date()]
        #         self.logger = Logger('results',[CSVOutputFormat])
        # self.reset()
        self._seed()

    def calculate_assets(self):

        assets = self.state[0]

        close_prices = self.state[1::(
            2 + len(self.tech_indicator_list))][:self.stock_dim]
        stocks_owned = self.state[2::(
            2 + len(self.tech_indicator_list))][:self.stock_dim]

        total_stock_value = np.sum(close_prices * stocks_owned)

        assets += total_stock_value
        return assets

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1
        if self.terminal:
            self.episode_end_date.append(self._get_date())
            self.handle_terminate()
        else:
            self.current_step_cost = 0
            actions = (actions * self.hmax).astype(int)

            begin_total_asset = self.calculate_assets()

            # argsort_actions = np.argsort(actions)
            sell_tickers = [self.tickers_list[i]
                            for i in np.where(actions < 0)[0]]
            buy_tickers = [self.tickers_list[i]
                           for i in np.where(actions > 0)[0]]

            for ticker in sell_tickers:
                action_value = actions[self.tickers_list.index(ticker)]
                # actions[self.tickers_list.index(ticker)] = self._sell_stock(
                #     ticker, action_value) * (-1)
                actions[self.tickers_list.index(ticker)] = self.trading_function.ordinary_sell_stock(
                    ticker, action_value) * (-1)

            for ticker in buy_tickers:
                action_value = actions[self.tickers_list.index(ticker)]
                # actions[self.tickers_list.index(ticker)] = self._buy_stock(
                #     ticker, action_value)
                actions[self.tickers_list.index(ticker)] = self.trading_function.ordinary_buy_stock(
                    ticker, action_value)

            self.actions_memory.append(actions)
            self.total_costs += self.current_step_cost
            self.timestep_since_last_trade += 1

            # state: s -> s+1
            self.day += 1
            self.data = self.df.loc[self.day, :]

            # self.state = self._update_state()
            self.state = self._update_state_hstack()

            end_total_asset = self.calculate_assets()
            self.caclulate_metrics(begin_total_asset, end_total_asset)

            self.asset_memory.append(end_total_asset)
            self.date_memory.append(self._get_date())
            self.reward = self.reward_function.compute_reward(
                begin_total_asset, end_total_asset)

            self.reward = float(self.reward)

            self.rewards_memory.append(self.reward)

            self.state_memory.append(
                self.state
            )  # add current state in state_recorder for each step

        return self.state.astype(np.float32), self.reward, self.terminal, False, {}

    def caclulate_metrics(self, begin_total_asset, end_total_asset):
        portfolio_return = (end_total_asset / begin_total_asset) - 1
        self.portfolio_return_memory.append(portfolio_return)

    def reset(
        self,
        *,
        seed=None,
        options=None,
    ):
        # initiate state
        self.episode_start_date.append(self._get_date())
        if len(self.episode_start_date) > 3:
            self.episode_start_date.pop(0)
        if len(self.episode_end_date) > 3:
            self.episode_end_date.pop(0)
        self.day = 0
        self.data = self.df.loc[self.day, :]
        self.state = self._initiate_state_hstack()
        self.stocks_owned = {ticker: initial_holding for ticker, initial_holding in zip(
            self.tickers_list, self.num_stock_shares)}

        self.current_step_cost = 0
        self.asset_memory = [
            self.initial_amount
            + np.sum(
                np.array(self.num_stock_shares)
                * np.array(self.state[1::(2 + len(self.tech_indicator_list))][:self.stock_dim])
            )
        ]
        # print("reset func - self.asset_memory:", self.asset_memory)
        self.portfolio_return_memory = [0]
        self.timestep_since_last_trade = 0
        self.total_costs = 0
        self.trades = 0
        self.terminal = False
        # self.iteration=self.iteration
        self.rewards_memory = []
        self.actions_memory = []
        self.date_memory = [self._get_date()]
        self.episode += 1

        return self.state.astype(np.float32), {}

    def calculate_sharpe_ratio(self):
        returns = np.array(self.portfolio_return_memory)
        return (np.mean(returns) / (np.std(returns) + 1e-9)) * np.sqrt(252)

    def render(self, mode="human", close=False):
        return self.state

    def _initiate_state_hstack(self):

        amount = np.array([self.initial_amount], dtype=np.float32)

        state_components = [amount]
        for idx, tic in enumerate(self.tickers_list):
            stock_data = self.data[self.data["tic"] == tic]
            if len(stock_data) == 0:
                print(f"No data found for ticker {tic} on date {self.day}")
            # print(f"initiate function: Stock data for {tic}:")
            # print("Initiate function . stockdata:", stock_data)
            close_price = np.array(
                [stock_data["close"].values[0]], dtype=np.float32)

            # Use the initial number of shares from self.num_stock_shares
            stock_owned = np.array(
                [self.num_stock_shares[idx]], dtype=np.float32)

            tech_indicators = [np.array(
                [stock_data[tech].values[0]], dtype=np.float32) for tech in self.tech_indicator_list]
            state_components.extend(
                [close_price, stock_owned] + tech_indicators)
        state_components.append(
            np.array([self.current_step_cost], dtype=np.float32))

        state_components.append(self._get_timestep_since_last_trade())
        state_components.append(self._get_number_of_trade_budget())
        state_components.append(self._get_number_of_done_trades())

        state = np.hstack(state_components, dtype=np.float32)

        return state

    def _update_state_hstack(self):
        current_amount = [self.state[0]]
        state_components = [current_amount]
        for tic in self.tickers_list:
            stock_data = self.data[(self.data["tic"] == tic)]
            if stock_data.empty:
                print(f"no data for ticker: {tic}")
                continue
            close_price = np.array(
                [stock_data["close"].values[0]], dtype=np.float32)
            stock_owned = np.array([self.stocks_owned[tic]], dtype=np.float32)
            tech_indicators = [np.array(
                [stock_data[tech].values[0]], dtype=np.float32) for tech in self.tech_indicator_list]
            tech_indicators = [np.array(
                [stock_data[tech].values[0]], dtype=np.float32) for tech in self.tech_indicator_list]
            state_components.extend(
                [close_price, stock_owned] + tech_indicators)
        state_components.append(
            np.array([self.current_step_cost], dtype=np.float32))

        state_components.append(self._get_timestep_since_last_trade())
        state_components.append(self._get_number_of_trade_budget())
        state_components.append(self._get_number_of_done_trades())

        state = np.hstack(state_components, dtype=np.float32)

        return state

    def _get_cash_balance(self):
        return np.array([self.state[0]])

    def _get_stock_prices(self):
        return self.state[1::(2 + len(self.tech_indicator_list))][:self.stock_dim]

    def _get_stock_owned(self):
        return self.state[2::(2 + len(self.tech_indicator_list))][:self.stock_dim]

    def _get_tech_indicators(self):
        tech_indicators = []
        for i in range(self.stock_dim):
            start_idx = 3 + i * (2 + len(self.tech_indicator_list))
            end_idx = start_idx + len(self.tech_indicator_list)
            tech_indicators.extend(self.state[start_idx:end_idx])
        return np.array(tech_indicators)

    def _get_current_step_cost(self):
        return np.array([self.current_step_cost])

    def calculate_state_shape(self):
        return sum(comp().shape[0] for comp in self.state_components)

    def get_current_share_holding(self, index):
        return self.state[2 + index * (2 + len(self.tech_indicator_list))]

    def _get_timestep_since_last_trade(self):
        return np.array([self.timestep_since_last_trade], dtype=np.float32)

    def _get_number_of_trade_budget(self):
        return np.array([self.number_of_trade_budget], dtype=np.float32)

    def _get_number_of_done_trades(self):
        return np.array([self.trades], dtype=np.float32)

    def get_closing_price(self, index):

        return self.state[1 + index * (2 + len(self.tech_indicator_list))]

    def update_num_shares(self, index, amount):
        tic = self.tickers_list[index]
        self.state[2 + index * (2 + len(self.tech_indicator_list))] += amount
        self.stocks_owned[tic] = self.state[2 +
                                            index * (2 + len(self.tech_indicator_list))]

    def _get_date(self):
        if len(self.df.tic.unique()) > 1:
            date = self.data.date.unique()[0]
        else:
            date = self.data.date
        return date


    def memory_collection(self):

        print("Date Memory Length:", len(self.date_memory))

        print("Asset Memory Length:", len(self.asset_memory))
        print("Portfolio Return Memory Length:",
              len(self.portfolio_return_memory))


        df_account_value = pd.DataFrame(
            {"date": self.date_memory, "account_value": self.asset_memory})
        df_return_memory = pd.DataFrame(
            {"date": self.date_memory, "daily_return": self.portfolio_return_memory})

        return df_account_value, df_return_memory

    def save_return_memory(self):

        df_return_memory = pd.DataFrame(
            {"date": self.date_memory, "daily_return": self.portfolio_return_memory})

        return df_return_memory

    def save_asset_memory(self):
        df_account_value = pd.DataFrame(
            {"date": self.date_memory, "account_value": self.asset_memory}
        )
        return df_account_value

    def handle_terminate(self):
        end_total_asset = self.calculate_assets()
        df_total_value = pd.DataFrame(self.asset_memory)
        tot_reward = end_total_asset - self.asset_memory[0]

        df_total_value.columns = ["account_value"]
        df_total_value["date"] = self.date_memory
        df_total_value["daily_return"] = df_total_value["account_value"].pct_change(
            1
        )

        annualized_return = (
            ((end_total_asset / self.asset_memory[0]) ** (365 / (self.day)))-1)*100

        if df_total_value["daily_return"].std() != 0:
            sharpe = (
                (252**0.5)
                * df_total_value["daily_return"].mean()
                / df_total_value["daily_return"].std()
            )
        df_rewards = pd.DataFrame(self.rewards_memory)
        df_rewards.columns = ["account_rewards"]
        df_rewards["date"] = self.date_memory[:-1]
        if self.episode % self.print_verbosity == 0:
            print(f"day: {self.day}, episode: {self.episode}")
            print(f"begin_total_asset: {self.asset_memory[0]:0.2f}")
            print(f"end_total_asset: {end_total_asset:0.2f}")
            print(f"total_reward: {tot_reward:0.2f}")
            print(f"total_cost: {self.total_costs:0.2f}")
            print(f"total_trades: {self.trades}")
            if df_total_value["daily_return"].std() != 0:
                print(f"Sharpe: {sharpe:0.3f}")
            # print(f"Equal wgt portfolio: {equal_weighted_ann_return:0.2f}%")
            print(f"Annual Return: {annualized_return:0.2f}%")
            print("=================================")

        if (self.model_name != "") and (self.mode != ""):
            df_actions = self.save_action_memory()
            df_actions.to_csv(
                "results/actions_{}_{}_{}.csv".format(
                    self.mode, self.model_name, self.iteration
                )
            )
            df_total_value.to_csv(
                "results/account_value_{}_{}_{}.csv".format(
                    self.mode, self.model_name, self.iteration
                ),
                index=False,
            )
            df_rewards.to_csv(
                "results/account_rewards_{}_{}_{}.csv".format(
                    self.mode, self.model_name, self.iteration
                ),
                index=False,
            )
            plt.plot(self.asset_memory, "r")
            plt.savefig(
                "results/account_value_{}_{}_{}.png".format(
                    self.mode, self.model_name, self.iteration
                )
            )
            plt.close()



        return self.state, self.reward, self.terminal, False, {}

    def save_action_memory(self):
        if len(self.df.tic.unique()) > 1:
            # date and close price length must match actions length
            date_list = self.date_memory[:-1]
            df_date = pd.DataFrame(date_list)
            df_date.columns = ["date"]

            action_list = self.actions_memory
            df_actions = pd.DataFrame(action_list)
            df_actions.columns = self.data.tic.values
            df_actions.index = df_date.date
            # df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        else:
            date_list = self.date_memory[:-1]
            action_list = self.actions_memory
            df_actions = pd.DataFrame(
                {"date": date_list, "actions": action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]
    

a2c_specific = {
    "lr": 0.0001,
    "gamma": 0.99,
    "use_gae": True,
    "lambda": 0.97,
    "num_workers": 1,
    "train_batch_size": 5000,
    "model": {
        "fcnet_hiddens": [256, 256],  # Size of the two hidden layers
        "fcnet_activation": "tanh",  # Activation function
        # ... add more model parameters if needed
    },
}

algo = a2c.A2C(env=StockTradingEnvAdvanced, config={
    # 'log_level': 'ERROR',
    'log_level': 'INFO',
    "env_config": {
         "hmax": 50000, 
    "initial_amount": 1000000, 
    "trading_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": indicators,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":1,
    "num_stock_shares": np.zeros(stock_dimension).tolist(),
    "df": train_df,  
    }, 
     
}, )


while True:
    result = algo.train()
    print(f"Mean Reward: {result['episode_reward_mean']}")



2023-08-28 01:20:15,933	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class '__main__.StockTradingEnvAdvanced'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class '__main__.StockTradingEnvAdvanced'>').build()` instead. This will raise an error in the future!
(pid=34723) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=34723) 2023-08-28 01:20:23,410	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=34723) 2023-08-28 01:20:23,410	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the future!
(RolloutWorker pid=34723) 2023-08-28 01:20:23,450	INFO policy.py:1294 -- Policy (worker

Mean Reward: nan
Mean Reward: nan
Mean Reward: nan


In [ ]:
# from ray.rllib.algorithms.a2c import A2C
# from env.multi_agent.manager import PortfolioManagerEnv
# from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
# from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
# from ray.rllib.policy.policy import Policy, PolicySpec

# from ray.tune.registry import register_env

# def env_creator(env_config):
#     return HierarchicalTradingEnv(...)

# register_env("combined_env", env_creator)

# manager_env = PortfolioManagerEnv()
# worker_env = StockTradingEnvAdvanced()



# policies = {
#     "manager_policy": PolicySpec(config={
#         "env_config": {
#             "n_stocks": stock_dimension, 
#         }
#     }),
#     "worker_policy": PolicySpec(config={
#         "env_config": {
#             "hmax": 50000, 
#             "initial_amount": 1000000, 
#             "trading_cost_pct": 0.001, 
#             "state_space": state_space, 
#             "stock_dim": stock_dimension, 
#             "tech_indicator_list": indicators,
#             "action_space": stock_dimension, 
#             "reward_scaling": 1e-4,
#             "print_verbosity":1,
#             "num_stock_shares": np.zeros(stock_dimension).tolist(),
#             "df": train_df,
#         }
#     }),
# }

# # Define the policy mapping function
# def policy_mapping_fn(agent_id, **kwargs):
#     if agent_id == "manager":
#         return "manager_policy"
#     else:
#         return "worker_policy"

# # Initialize the A2C algorithm with the multi-agent configuration
# algo = A2C(env="combined_env", config={
#     "multiagent": {
#         "policies": policies,
#         "policy_mapping_fn": policy_mapping_fn,
#     },
# })

# while True:
#     print(algo.train())


In [ ]:
from ray.rllib.algorithms.a2c import A2C
from ray.rllib.algorithms.ppo import PPO
from env.multi_agent.manager import PortfolioManagerEnv
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from ray.rllib.policy.policy import Policy, PolicySpec

from ray.tune.registry import register_env

def env_creator(env_config):
    return HierarchicalTradingEnv(manager_config=env_config["manager_config"],
                                  worker_config=env_config["worker_config"])

register_env("combined_env", env_creator)

policies = {
    "manager_policy": PolicySpec(config={
        "env_config": {
            "n_stocks": stock_dimension, 
        }
    }),
    "worker_policy": PolicySpec(config={
        "env_config": {
             "hmax": 50000, 
            "initial_amount": 1000000, 
            "trading_cost_pct": 0.001, 
            "state_space": state_space, 
            "stock_dim": stock_dimension, 
            "tech_indicator_list": indicators,
            "action_space": stock_dimension, 
            "reward_scaling": 1e-4,
            "print_verbosity":1,
            "num_stock_shares": np.zeros(stock_dimension).tolist(),
            "df": train_df,
        }
    }),
}

# Define the policy mapping function
def policy_mapping_fn(agent_id, **kwargs):
    if agent_id == "manager":
        return "manager_policy"
    else:
        return "worker_policy"

# Initialize the A2C algorithm with the multi-agent configuration
algo = PPO(env="combined_env", config={
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env_config": {
        "manager_config": {
            "n_stocks": stock_dimension,
        },
        "worker_config": {
             "hmax": 50000, 
            "initial_amount": 1000000, 
            "trading_cost_pct": 0.001, 
            "state_space": state_space, 
            "stock_dim": stock_dimension, 
            "tech_indicator_list": indicators,
            "action_space": stock_dimension, 
            "reward_scaling": 1e-4,
            "print_verbosity":1,
            "num_stock_shares": np.zeros(stock_dimension).tolist(),
            "df": train_df,
        }
    }
})

manager_config = {"n_stocks": stock_dimension}
worker_config = {
     "hmax": 50000, 
            "initial_amount": 1000000, 
            "trading_cost_pct": 0.001, 
            "state_space": state_space, 
            "stock_dim": stock_dimension, 
            "tech_indicator_list": indicators,
            "action_space": stock_dimension, 
            "reward_scaling": 1e-4,
            "print_verbosity":1,
            "num_stock_shares": np.zeros(stock_dimension).tolist(),
            "df": train_df,
}


# env = HierarchicalTradingEnv(manager_config=manager_config, worker_config=worker_config)


while True:
    print(algo.train())

2023-08-30 16:25:49,697	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-30 16:25:49,697	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='combined_env', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('combined_env').build()` instead. This will raise an error in the future!
(pid=34389) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=34389) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=34389, ip=127.0.0.1, actor_id=f65a

TypeError: HierarchicalTradingEnv.__init__() got an unexpected keyword argument 'manager_config'

In [ ]:
from ray.rllib.algorithms.a2c import A2C
from env.multi_agent.manager import PortfolioManagerEnv
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from ray.rllib.policy.policy import Policy, PolicySpec
manager_config = {"n_stocks": stock_dimension}
worker_config = {
     "hmax": 50000, 
            "initial_amount": 1000000, 
            "trading_cost_pct": 0.001, 
            "state_space": state_space, 
            "stock_dim": stock_dimension, 
            "tech_indicator_list": indicators,
            "action_space": stock_dimension, 
            "reward_scaling": 1e-4,
            "print_verbosity":1,
            "num_stock_shares": np.zeros(stock_dimension).tolist(),
            "df": train_df,
}


env = HierarchicalTradingEnv(manager_config=manager_config, worker_config=worker_config)

TypeError: HierarchicalTradingEnv.__init__() got an unexpected keyword argument 'manager_config'

In [ ]:
env = HierarchicalTradingEnv(manager_config=manager_config, worker_config=worker_config)
obs = env.reset()
dummy_action = {
    "manager": np.zeros(env.action_space["manager"].shape),
    "worker": np.zeros(env.action_space["worker"].shape)
}
obs, reward, done, info = env.step(dummy_action)
print(obs, reward, done, info)

TypeError: HierarchicalTradingEnv.__init__() got an unexpected keyword argument 'manager_config'

In [ ]:
from ray.rllib.algorithms.a2c import A2C
from env.multi_agent.manager import PortfolioManagerEnv
from env.multi_agent.HierarchicalTradingEnv import HierarchicalTradingEnv
from env.env_ST_advanced_raylib import StockTradingEnvAdvanced
from ray.rllib.policy.policy import Policy, PolicySpec
df = pd.read_pickle("pharma.pkl")
df = df[df['tic'] != 'NVO.US']
df = df[df['tic'] != 'DHR.US']
df

train_start = "1986-01-02"
train_end = "2010-12-31"
validate_start = "2011-01-01"
validate_end = "2015-12-31"
test_start = "2016-01-01"
test_end = "2023-08-17"


indicators = [ 'avgvol_50',
       'sma_10', 'sma_50', 'sma_100', 'sma_200', 'wma_50', 'rsi_14',
       'volatility_30', 'volatility_100', 'stddev_30', 'dmi_14', 'adx_14',
       'macd', 'atr_14',"volume"]

def data_split(df, start, end, target_date_col="date"):
    data = df[(df[target_date_col] >= start) & (df[target_date_col] < end)]
    data = data.sort_values([target_date_col, "tic"], ignore_index=True)
    data.index = data[target_date_col].factorize()[0]
    return data
train_df = data_split(df, train_start, train_end)
validate_df = data_split(df, validate_start, validate_end)
test_df = data_split(df, test_start, test_end)

stock_dimension = len(train_df.tic.unique())
state_space = 1 + 2*stock_dimension + len(indicators)*stock_dimension

# HierarchicalTradingEnv configuration
env_config = {
    "manager_config": {
        "n_stocks": stock_dimension
    },
    "worker_config": {
        "hmax": 50000,
        "initial_amount": 1000000,
        "trading_cost_pct": 0.001,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": indicators,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4,
        "print_verbosity": 1,
        "num_stock_shares": np.zeros(stock_dimension).tolist(),
        "df": train_df
    }
}

# PPO configuration
a2c_config = {
    "multiagent": {
        "policies": {
            "manager_policy": PolicySpec(config={
                "env_config": {
                    "n_stocks": stock_dimension, 
                }
            }),
            "worker_policy": PolicySpec(config={
                "env_config": env_config["worker_config"]
            })
        },
        "policy_mapping_fn": lambda agent_id: "manager_policy" if agent_id == "manager" else "worker_policy",
    },
    "env_config": env_config
}

# Initialize the PPO algorithm with the HierarchicalTradingEnv
algo = A2C(env=HierarchicalTradingEnv, config=a2c_config)

# Training loop
while True:
    result = algo.train()
    print(f"Mean Reward: {result['episode_reward_mean']}")

2023-08-30 16:27:44,553	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class 'env.multi_agent.HierarchicalTradingEnv.HierarchicalTradingEnv'>').build()` instead. This will raise an error in the future!
(pid=34862) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=34862) 2023-08-30 16:27:50,795	WARNING env.py:298 -- Your MultiAgentEnv <HierarchicalTradingEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
(RolloutWorker pid=34862) 2023-08-30 16:27:50,839	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in

RayTaskError(TypeError): [36mray::RolloutWorker.apply()[39m (pid=34862, ip=127.0.0.1, actor_id=3d6fd106e9fb114e86e08d0601000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x162edfa90>)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 185, in apply
    raise e
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/actor_manager.py", line 176, in apply
    return func(self, *args, **kwargs)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/execution/rollout_ops.py", line 86, in <lambda>
    lambda w: w.sample(), local_worker=False, healthy_only=True
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 696, in sample
    batches = [self.input_reader.next()]
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rl

In [ ]:
tune.run(
    "A2C",
    stop={"training_iteration": 500},  # Stop after 100 training iterations.
    config={
        "log_level": 'ERROR',
        "env": StockTradingEnvAdvanced,
        "env_config": {
             "hmax": 50000, 
    "initial_amount": 1000000, 
    "trading_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": indicators,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":1,
    "num_stock_shares": np.zeros(stock_dimension).tolist(),
    "df": train_df,  
        },
        # Any other configurations for the algorithm can be added here.
    },
    checkpoint_freq=10,  # Save a checkpoint every 10 iterations.
    checkpoint_at_end=True,  # Save a checkpoint at the end of training.
    local_dir="/Users/floriankockler/rayresults/tune/",  # Directory to save the checkpoints.
)

2023-08-29 00:52:52,210	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(pid=37625) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(A2C pid=37625) 2023-08-29 00:52:57,608	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
(A2C pid=37625) 2023-08-29 00:52:57,608	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!
(A2C pid=37625) 2023-08-29 00:52:57,608	WARNING algorithm_config.py:656 -- Cannot create A2CConfig from given `config_dict`! Property __stdout_file__ not supported.
(pid=37633) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(pid=37634) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=37633) /Users/floriankockler/anaconda3/envs/py310/lib/p

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
A2C_StockTradingEnvAdvanced_9e6c6_00000,29344,"{'ObsPreprocessorConnector_ms': 0.005626678466796875, 'StateBufferConnector_ms': 0.002658367156982422, 'ViewRequirementAgentConnector_ms': 0.06091594696044922}","{'num_env_steps_sampled': 29344, 'num_env_steps_trained': 29344, 'num_agent_steps_sampled': 29344, 'num_agent_steps_trained': 29344}",{},6305,{},-586.163,-1409.08,-1801.61,0,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 40.0, 'cur_lr': 0.0001, 'entropy_coeff': 0.01, 'policy_entropy': 600.8357543945312, 'policy_loss': -7689.1181640625, 'vf_loss': 4058.12646484375}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 32.0, 'num_grad_updates_lifetime': 917.0, 'diff_num_grad_updates_vs_sampler_policy': 916.0}}, 'num_env_steps_sampled': 29344, 'num_env_steps_trained': 29344, 'num_agent_steps_sampled': 29344, 'num_agent_steps_trained': 29344}",29344,29344,29344,2688,265.839,29344,2688,265.839,0,2,0,0,2688,"{'cpu_util_percent': 41.059999999999995, 'ram_util_percent': 80.01333333333334}",{},{},{},"{'mean_raw_obs_processing_ms': 0.19120862149607865, 'mean_inference_ms': 0.46720135257532813, 'mean_action_processing_ms': 0.08567972774585784, 'mean_env_wait_ms': 6.3043421187907, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -586.1628444378372, 'episode_reward_min': -1801.6058580228457, 'episode_reward_mean': -1409.0808862697997, 'episode_len_mean': 6305.0, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-1801.6058580228457, -1737.9959105325022, -1510.558932086014, -586.1628444378372], 'episode_lengths': [6305, 6305, 6305, 6305]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.19120862149607865, 'mean_inference_ms': 0.46720135257532813, 'mean_action_processing_ms': 0.08567972774585784, 'mean_env_wait_ms': 6.3043421187907, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.005626678466796875, 'StateBufferConnector_ms': 0.002658367156982422, 'ViewRequirementAgentConnector_ms': 0.06091594696044922}}","{'training_iteration_time_ms': 117.255, 'sample_time_ms': 113.876, 'load_time_ms': 0.153, 'load_throughput': 208736.747, 'learn_time_ms': 2.136, 'learn_throughput': 14979.824, 'synch_weights_time_ms': 1.048}"


(RolloutWorker pid=37633) day: 6304, episode: 5
(RolloutWorker pid=37633) begin_total_asset: 1000000.00
(RolloutWorker pid=37633) end_total_asset: 5432771.50
(RolloutWorker pid=37633) total_reward: 4432771.50
(RolloutWorker pid=37633) total_cost: -13389194.87
(RolloutWorker pid=37633) total_trades: 47562
(RolloutWorker pid=37633) Sharpe: 0.422
(RolloutWorker pid=37633) Annual Return: 10.30%
(RolloutWorker pid=37633) =================================
(RolloutWorker pid=37633) day: 6304, episode: 6 [repeated 2x across cluster]
(RolloutWorker pid=37633) begin_total_asset: 1000000.00 [repeated 2x across cluster]
(RolloutWorker pid=37633) end_total_asset: 3132257.75 [repeated 2x across cluster]
(RolloutWorker pid=37633) total_reward: 2132257.75 [repeated 2x across cluster]
(RolloutWorker pid=37633) total_cost: -9584600.84 [repeated 2x across cluster]
(RolloutWorker pid=37633) total_trades: 40406 [repeated 2x across cluster]
(RolloutWorker pid=37633) Sharpe: 0.315 [repeated 2x across cluster

In [ ]:
env_test = HierarchicalTradingEnv(manager_config=manager_config, worker_config=worker_config)
obs_test = env_test.reset()
action_test = {"manager": np.zeros(env_test.action_space["manager"].shape), "worker": np.zeros(env_test.action_space["worker"].shape)}
obs_next, rewards, dones, infos = env_test.step(action_test)
print(obs_next, rewards, dones, infos)

for i in range(100):
    action_test = {
        "manager": env_test.action_space["manager"].sample(),
        "worker": env_test.action_space["worker"].sample()
    }
    obs_next, rewards, dones, infos = env_test.step(action_test)
    if dones['__all__']:
        env_test.reset()

print("Test completed successfully!")

TypeError: HierarchicalTradingEnv.__init__() got an unexpected keyword argument 'manager_config'